# Database Cleaning

The survey was applied in a Google Form. The data was exported to the CSV file 'Ecological Inteligence.csv'. We cleaned the data with Python because it was easier to manipulate the data than in R.

First, we download the CSV file from Google Forms: [link to the file](https://docs.google.com/forms/d/1Fsd6EXfChcF_qHH-MbgcP4FffkvTIyjmq9c837x7HQY/edit).

The following code cleans the data. We did the following steps:
1. Read the CSV file.
2. Transform the data to a pandas DataFrame.
3. Do a first cleaning of the column 'program'. It still needs to properly classify all program descriptions.
4. Clean the column 'nationality'.
5. Code all the responses of the Likert scale to 1 to 5.

In [3]:
import pandas as pd

path = './../data/'
dfenc = pd.read_csv(path + 'Ecological Inteligence Scale 2.0.csv')
dfenc.columns = [
    "date", "student", "programa", "campus", "sex", "nationality", "age", "holis1", "holis2", "holis3", "holis4", "holis5", 
    "social1", "social2", "social3", "social4", "econ1", "econ2", "econ3", "change", "argument"
]
dfenc.tail(3)

,date,student,programa,campus,sex,nationality,age,holis1,holis2,holis3,...,holis5,social1,social2,social3,social4,econ1,econ2,econ3,change,argument
318,2025/06/04 11:31:58 AM MDT,a00833439@tec.mx,LAET,Campus Monterrey,Mujer,Mexicana,21 a 22 años,Partly agree,Not sure,Completely agree,...,Partly agree,Completely agree,Not sure,Partly agree,Partly agree,Partly agree,Partly agree,Partly agree,9,Al principio pensaba que era solo un tema de c...
319,2025/06/04 11:34:24 AM MDT,a01284919@tec.mx,LAET,Campus Monterrey,Mujer,Mexicana,21 a 22 años,Completely agree,Partly agree,Completely agree,...,Completely agree,Completely agree,Completely agree,Completely agree,Partly agree,Not sure,Not sure,Partly agree,9,"Gracias a los modelos de Triple Bottom Line, C..."
320,2025/06/04 11:46:13 AM MDT,a01246690@tec.mx,LAET,Campus Monterrey,Hombre,México,23 a 24 años,Completely agree,Not sure,Completely agree,...,Not sure,Not sure,Completely disagree,Partly agree,Partly agree,Partly agree,Partly agree,Partly agree,10,"Considero que es un tema recurrente, y de gran..."


Transform date to datetime.

In [4]:
from datetime import datetime

# Remove timezone info and convert to datetime
dfenc['date'] = dfenc['date'].str.replace(r' [A-Z]{2,}$', '', regex=True)
dfenc['date'] = pd.to_datetime(dfenc['date'], format='%Y/%m/%d %I:%M:%S %p')

records_per_date = dfenc['date'].dt.date.value_counts().sort_index()
records_per_date_df = records_per_date.reset_index()
records_per_date_df.columns = ['date', 'count']
records_per_date_df

,date,count
0,2025-05-06,46
1,2025-05-08,10
2,2025-05-12,20
3,2025-05-14,17
4,2025-05-15,24
5,2025-05-19,17
6,2025-05-20,53
7,2025-05-22,21
8,2025-05-23,58
9,2025-05-24,2


In [5]:
dfenc['programa'] = dfenc['programa'].str.strip().replace(r'\s+', ' ', regex=True)
dfenc['programa'] = dfenc['programa'].apply(lambda x: 'LAF' if isinstance(x, str) and x.lower() == 'laf' else x)
dfenc['programa'] = dfenc['programa'].apply(lambda x: 'LAET' if isinstance(x, str) and x.lower() == 'laet' else x)
programa_freq = dfenc['programa'].value_counts().reset_index()
programa_freq.columns = ['programa', 'frequency']
programa_freq

,programa,frequency
0,LAET,151
1,LAF,95
2,LAE,27
3,Finanzas,9
4,Estrategia y Transformación de Negocios,8
5,Licenciatura en Finanzas,7
6,LAET19,3
7,LAET con doble titulación en LIN,2
8,LAF y LIN,1
9,LAE19 Y Master in International Management by ...,1


In [6]:
dfenc['campus'] = dfenc['campus'].str.replace('Campus', '', regex=False).str.strip()
campus_freq = dfenc['campus'].value_counts().reset_index()
campus_freq.columns = ['campus', 'frequency']
campus_freq


,campus,frequency
0,Guadalajara,107
1,Monterrey,53
2,San Luis Potosí,38
3,Sinaloa,30
4,Aguascalientes,26
5,Toluca,22
6,Sonora Norte,17
7,Laguna,12
8,Chihuahua,8
9,León,8


In [7]:
sex_freq = dfenc['sex'].value_counts().reset_index()
sex_freq.columns = ['sex', 'frequency']
sex_freq

,sex,frequency
0,Mujer,165
1,Hombre,156


In [12]:
from difflib import SequenceMatcher

nationstr = dfenc['nationality']
similarity_scores = nationstr.apply(lambda x: SequenceMatcher(None, str(x).strip().lower(), 'mexicana').ratio())
similarity_table = pd.DataFrame({'nationality': nationstr, 'similarity_to_mexicana': similarity_scores})
dfenc.loc[similarity_scores >= 0.875, 'nationality'] = 'Mexicana'
dfenc.loc[dfenc['nationality'].str.strip().str.lower().isin(['mexico', 'méxico', 'mex']), 'nationality'] = 'Mexicana'
dfenc.loc[dfenc['nationality'].str.contains('Estados Unidos|Americana', case=False, na=False), 'nationality'] = 'Estadounidense'
dfenc.loc[dfenc['nationality'].str.contains('Estadounidense', case=False, na=False), 'nationality'] = 'Estadounidense'
dfenc.loc[dfenc['nationality'].str.strip().str.lower() == 'colombia', 'nationality'] = 'Colombiana'
dfenc.loc[dfenc['nationality'].str.strip().str.lower() == 'chileno', 'nationality'] = 'Chilena'
nationality_freq = dfenc['nationality'].value_counts().reset_index()
nationality_freq.columns = ['nationality', 'frequency']
print(nationality_freq)

      nationality  frequency
0        Mexicana        309
1  Estadounidense          5
2     Ecuatoriana          2
3         Chilena          1
4     Salvadoreña          1
5       Holandesa          1
6      Colombiana          1
7      Canadiense          1


In [13]:
age_freq = dfenc['age'].value_counts().reset_index()
age_freq.columns = ['age', 'frequency']
age_freq

,age,frequency
0,21 a 22 años,257
1,23 a 24 años,56
2,25 años o más,8


In [14]:
response_map = {
    'Completely agree': 5,
    'Partly agree': 4,
    'Not sure': 3,
    'Partly disagree': 2,
    'Completely disagree': 1
}
dfenc.loc[:, 'holis1': 'econ3'] = dfenc.loc[:, 'holis1': 'econ3'].replace(response_map)
dfenc.loc[:,'holis1':'econ3']

C:\Users\L01032102\AppData\Local\Temp\ipykernel_22928\1948386698.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfenc.loc[:, 'holis1': 'econ3'] = dfenc.loc[:, 'holis1': 'econ3'].replace(response_map)


,holis1,holis2,holis3,holis4,holis5,social1,social2,social3,social4,econ1,econ2,econ3
0,4,5,5,5,5,5,5,5,5,5,5,5
1,5,4,5,4,5,4,2,5,5,4,5,4
2,5,2,4,5,5,5,5,3,4,4,5,4
3,4,2,5,3,4,5,3,3,4,5,5,4
4,5,4,4,5,4,5,3,4,4,4,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...
316,5,4,5,4,4,4,3,5,4,4,4,4
317,3,3,4,3,3,3,4,3,4,3,4,4
318,4,3,5,2,4,5,3,4,4,4,4,4
319,5,4,5,4,5,5,5,5,4,3,3,4


## Save databse

Save clean database to a file.

In [15]:
dfenc.to_excel(path + 'Eco_Intel2_clean.xlsx', index=False)